# Dokumentation der Tags
## s[]s	Stichwort
## p[]p	Person
Für die Ansetzung von Personennamen kommen folgende Stichwortteile bzw. Ordnungshilfen in Frage:
1. Name, 
2. Vorname(n), 
3. Adelstitel/Herrschaftsbezeichnung, 
4. Akademischer Grad, 
5. Beruf, 
6. Ort
7. Sonstiges
8. Präfixe vor Personennamen
9. Pseudonym / angenommene Namen
Beispiel:
- der Aktentitel enthält:
... der Prager Buchhändler J. Gelder ...
- in Kat. 40 wird aufgenommen:
... der p[6{Prag}er 5{Buchhändler} 2{J}.{<ohannes>} 1{Gelder}]p ...
Im Personenregister entsteht daraus:
Gelder, Johannes <Buchhändler in Prag>
Zusätzlich werden Berufe und Orte indiziert, d.h. verklammerte Indizierung: 
der p[6{o[Prag]o}er 5{s[Buchhändler]s} 2{J<ohannes>}. 1{Gelder}]p

Namen von Adeligen/Königen/Herzögen usw. werden nur unter dem Vornamen in Klammer 2 indiziert. Bekannte Zusätze werden in <> eingefügt.


## k[]k	Körperschaft
## o[]o	Ortsregister
## t[]t	Titel
## z[]z	Zeit



In [1]:
import json
import re

In [154]:
with open("../json/2021-03-16-inventar-gesamt.json", 'r') as f:
    db = json.load(f)

In [183]:
def re_parse(type, string):
    # print(f"type {type} und string {string}\n")
    ergebnis_liste = list()
    if substring_re_ergebnisse := re.finditer(r"([a-z])\[(.+?)\]\1", string):
        for substring_match in substring_re_ergebnisse:
            ergebnis_liste.extend(re_parse(substring_match.group(1), substring_match.group(2)))
    match type:
        case "p":
            person = dict()
            string = re.sub(r"([sop]\[)|(\][sop])","",string)
            if re_matches := re.findall(r"([1-9])\{([^\{]+)}", string):
                for re_match in re_matches:
                    match re_match[0]:
                        case "1":
                            person["name"] = re_match[1]
                        case "2":
                            person["vorname"] = re_match[1]
                        case "3":
                            person["adelstitel"] = re_match[1]
                        case "4":
                            person["akademischergrad"] = re_match[1]
                        case "5":
                            person["beruf"] = re_match[1]
                        case "6":
                            person["ort"] = re_match[1]
                        case "7":
                            person["sonstiges"] = re_match[1]
                        case "8":
                            person["praefix"] = re_match[1]
                        case "9":
                            person["pseudonym"] = re_match[1]
            else:
                person["name"] = string
            ergebnis_liste.append({'p':person})
            return ergebnis_liste

        case "s":
            string = re.sub(r"([tksopz]\[)|(\][tksopz])","",string)
            if re_matches := re.findall(r"([1-9])\{([^\{]+)}", string):
                schlagwort = dict()
                for re_match in re_matches:
                    match re_match[0]:
                        case "1":
                            schlagwort["1"] = re_match[1]
                        case "2":
                            schlagwort["2"] = re_match[1]
                        case "3":
                            schlagwort["3"] = re_match[1]
            else:
                schlagwort = string
            ergebnis_liste.append({'s':schlagwort})
            return ergebnis_liste

        case "k":
            string = re.sub(r"([sopt]\[)|(\][sopt])","",string)
            string = re.sub(r"([1-9]\{)|(\})", "", string)
            if re_matches := re.findall(r"([1-9])\{([^\{]+)}", string):
                koerperschaft = dict()
                for re_match in re_matches:
                    match re_match[0]:
                        case "1":
                            koerperschaft["1"] = re_match[1]
                        case "2":
                            koerperschaft["2"] = re_match[1]
            else:
                koerperschaft = string

            ergebnis_liste.append({'k':koerperschaft})
            return ergebnis_liste
            
        case "o":
            if re_matches := re.findall(r"([1-9])\{([^\{]+)}", string):
                ort = dict()
                for re_match in re_matches:
                    match re_match[0]:
                        case "1":
                            ort["1"] = re_match[1]
                        case "2":
                            ort["2"] = re_match[1]
                        case "3":
                            ort["3"] = re_match[1]
            else:
                ort = string
            ergebnis_liste.append({'o': ort})
            return ergebnis_liste
        case "t":
            titel = re.sub(r"([tksopz]\[)|(\][tksopz])","",string)
            titel = re.sub(r"([1-9]\{)|(\})", "", titel)
            ergebnis_liste.append({'t':titel})
            return ergebnis_liste
        case "z":
            return ergebnis_liste

In [184]:
def write_faszikellemmata(sig, faszikel_lemmata):
    for lemma in faszikel_lemmata:
        match list(lemma.keys())[0]:
            case 'p':
                person_format = ""
                if "name" in lemma['p']:
                    person_format = lemma['p']["name"]
                    if "vorname" in lemma['p']:
                        person_format += f', {lemma["p"]["vorname"]}'
                    if "praefix" in lemma['p']:
                        person_format += f' {lemma["p"]["praefix"]}'
                    if "akademischergrad" in lemma['p']:
                        person_format += f' ({lemma["p"]["akademischergrad"]})'
                if "adelstitel" in lemma['p'] and "beruf" not in lemma['p']:
                    if "ort" in lemma['p']:
                        person_format += f'{lemma["p"]["ort"]}, '
                    person_format += f'{lemma["p"]["adelstitel"]} '
                    if "vorname" in lemma['p']:
                        person_format += f'{lemma["p"]["vorname"]} '
                    if "name" in lemma['p']:
                        person_format += f'{lemma["p"]["name"] } '
                    if "praefix" in lemma['p']:
                        person_format += f'{lemma["p"]["praefix"]}'
                elif "beruf" in lemma['p'] or "ort" in lemma['p'] or "pseudonym" in lemma['p'] or "sonstiges" in lemma['p']:
                    person_format += ' <'
                    if "beruf" in lemma['p'] and "ort" in lemma['p']:
                        person_format += f'{lemma["p"]["beruf"]} in {lemma["p"]["ort"]}'
                    elif "beruf" in lemma['p'] and "ort" not in lemma['p']:
                        person_format += f"{lemma['p']['beruf']}"
                    elif "ort" in lemma['p'] and "beruf" not in lemma['p']:
                        person_format += lemma['p']["ort"]
                    if "adelstitel" in lemma['p']:
                        person_format += lemma['p']['adelstitel']
                    if "sonstiges" in lemma['p']:
                        person_format += f" {lemma['p']['sonstiges']}"
                    person_format += '>'
                
                with open('personen.csv', 'a') as p:
                    p.write(f'"{person_format}","{sig}"\n')
            
            case 't':
                with open('titel.csv', 'a') as t:
                    t.write(f'"{lemma['t']}","{sig}"\n')
            
            case 's':
                if isinstance(lemma['s'], dict):
                    schlagwort_format = str()
                    if "1" in lemma['s']:
                        schlagwort_format = lemma['s']['1']
                    if "2" in lemma['s']:
                        schlagwort_format += f", {lemma['s']['2']}"
                    if "3" in lemma['s']:
                        schlagwort_format += f", {lemma['s']['3']}"
                else:
                    schlagwort_format = lemma['s']

                with open('schlagworte.csv', 'a') as s:
                    s.write(f'"{schlagwort_format}","{sig}"\n')
            case 'o':
                if isinstance(lemma['o'], dict):
                    ort_format = str()
                    if "1" in lemma['o']:
                        ort_format = lemma['o']['1']
                    if "2" in lemma['o']:
                        ort_format += f", {lemma['o']['2']}"
                    if "3" in lemma['o']:
                        ort_format += f", {lemma['o']['3']}"
                else:
                    ort_format = lemma['o']
                with open('orte.csv', 'a') as o:
                    o.write(f'"{ort_format}","{sig}"\n')
            
            case 'k':
                if isinstance(lemma['k'], dict):
                    koerperschaft_format = str()
                    if "1" in lemma['k']:
                        koerperschaft_format = lemma['k']['1']
                    if "2" in lemma['k']:
                        koerperschaft_format += f", {lemma['k']['2']}"
                else:
                    koerperschaft_format = lemma['k']
                with open('koerperschaften.csv', 'a') as k:
                    k.write(f'"{koerperschaft_format}","{sig}"\n')

In [185]:
# print(f'{len(db["archive"])} Archive')
lemmata = list()
for archiv in db["archive"].items():
    # print(archiv)
    for bestand in archiv[1]["bestaende"].items():
        # print(bestand)
        if "unterbestaende" in bestand[1]:
            for unterbestand in bestand[1]["unterbestaende"].items():
                if "faszikel" in unterbestand[1]:
                    for faszikel in unterbestand[1]["faszikel"].items():
                        faszikel_lemmata = list()
                        sig = f"{archiv[0]} / {bestand[0]} / {unterbestand[0]} / {faszikel[0]}"
                        for kategorie in faszikel[1].items():
                            if re_ergebnisse := re.finditer(r"([a-z])\[(.+?)\]\1", kategorie[1]):
                                for match in re_ergebnisse:
                                    faszikel_lemmata.extend(re_parse(match.group(1), match.group(2)))
                        write_faszikellemmata(sig, faszikel_lemmata)
        elif "faszikel" in bestand[1]:
            for faszikel in bestand[1]["faszikel"].items():
                faszikel_lemmata = list()
                sig = f"{archiv[0]} / {bestand[0]} / {faszikel[0]}"
                for kategorie in faszikel[1].items():
                    if re_ergebnisse := re.finditer(r"([a-z])\[(.+?)\]\1", kategorie[1]):
                        for match in re_ergebnisse:
                            faszikel_lemmata.extend(re_parse(match.group(1), match.group(2)))
                write_faszikellemmata(sig, faszikel_lemmata)